In [4]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt


# Define directories
train_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Train'
test_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Test'

# Data augmentation and preprocessing for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for validation data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # Use 'binary' for two classes: fresh and rotten
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,  # No labels for test data
    shuffle=False
)


In [5]:
# Define directories
train_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Train'
test_dir = 'D:/windsor/Sem-2/ASP/Project/Project/dataset/Test'

# Data augmentation and preprocessing for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for validation data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'  # Use 'binary' for two classes: fresh and rotten
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,  # No labels for test data
    shuffle=False
)


Found 23619 images belonging to 18 classes.
Found 6738 images belonging to 14 classes.


In [6]:
# Load VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10
)


Epoch 1/10


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


738/738 ━━━━━━━━━━━━━━━━━━━━ 3410s 5s/step - accuracy: 0.1039 - loss: -4629.5732
Epoch 2/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 4s 61us/step - accuracy: 0.1250 - loss: -36478.6211 
Epoch 3/10


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


738/738 ━━━━━━━━━━━━━━━━━━━━ 3416s 5s/step - accuracy: 0.1037 - loss: -50145.6016
Epoch 4/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 5s 11us/step - accuracy: 0.0938 - loss: -96167.8359 
Epoch 5/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 3550s 5s/step - accuracy: 0.1047 - loss: -141631.1406
Epoch 6/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 5s 11us/step - accuracy: 0.0625 - loss: -256551.2969 
Epoch 7/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 3541s 5s/step - accuracy: 0.1056 - loss: -274170.8438
Epoch 8/10
738/738 ━━━━━━━━━━━━━━━━━━━━ 5s 15us/step - accuracy: 0.1250 - loss: -370199.2500 
Epoch 9/10
421/738 ━━━━━━━━━━━━━━━━━━━━ 24:56 5s/step - accuracy: 0.1025 - loss: -419159.5938

KeyboardInterrupt: 

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()
